In [1]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from tqdm import tqdm
import shutil
#---------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
#---------------------------------------
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
from sklearn.model_selection import KFold
#---------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [2]:
def move_files(file_paths, labels, train_dirs):
    with tqdm(total=len(file_paths), desc="Moving Files", bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt}") as pbar:
        for file_path, label in zip(file_paths, labels):
            if label == 0:  # bottle
                target_dir = train_dirs[0]
            elif label == 1:  # canister
                target_dir = train_dirs[1]
            elif label == 2:  # cans
                target_dir = train_dirs[2]
            elif label == 3:  # cardboard
                target_dir = train_dirs[3]
            elif label == 4:  # detergent
                target_dir = train_dirs[4]
            
            # Copy the file to the target directory
            shutil.copy(file_path, target_dir)
            pbar.update(1)  # Update progress bar by 1 step



def data_prep(input_dir, output_dir,i):

    # Define output directories for each class in the train set
    train_dir = os.path.join(output_dir, i)
    train_bottle_dir = os.path.join(train_dir, 'bottle')
    train_canister_dir = os.path.join(train_dir, 'canister')
    train_cans_dir = os.path.join(train_dir, 'cans')
    train_cardboard_dir = os.path.join(train_dir, 'cardboard')
    train_detergent_dir = os.path.join(train_dir, 'detergent')

    # Create the directories if they don't exist
    os.makedirs(train_bottle_dir, exist_ok=True)
    os.makedirs(train_canister_dir, exist_ok=True)
    os.makedirs(train_cans_dir, exist_ok=True)
    os.makedirs(train_cardboard_dir, exist_ok=True)
    os.makedirs(train_detergent_dir, exist_ok=True)

    # Collect images from subdirectories under the bottle class
    bottle_dir = os.path.join(input_dir, 'bottle')
    bottle_images = []
    for subdir in os.listdir(bottle_dir):
        subdir_path = os.path.join(bottle_dir, subdir)
        if os.path.isdir(subdir_path):  # Ensure it's a directory
            bottle_images.extend([os.path.join(subdir_path, img) for img in os.listdir(subdir_path)])

    # Repeat for other categories
    canister_dir = os.path.join(input_dir, 'canister/canister')
    canister_images = [os.path.join(canister_dir, img) for img in os.listdir(canister_dir) if os.path.isfile(os.path.join(canister_dir, img))]

    cans_dir = os.path.join(input_dir, 'cans/cans')
    cans_images = [os.path.join(cans_dir, img) for img in os.listdir(cans_dir) if os.path.isfile(os.path.join(cans_dir, img))]

    cardboard_dir = os.path.join(input_dir, 'cardboard')
    cardboard_images = []
    for subdir in os.listdir(cardboard_dir):
        subdir_path = os.path.join(cardboard_dir, subdir)
        if os.path.isdir(subdir_path):  # Ensure it's a directory
            cardboard_images.extend([os.path.join(subdir_path, img) for img in os.listdir(subdir_path)])

    detergent_dir = os.path.join(input_dir, 'detergent')
    detergent_images = []
    for subdir in os.listdir(detergent_dir):
        subdir_path = os.path.join(detergent_dir, subdir)
        if os.path.isdir(subdir_path):  # Ensure it's a directory
            detergent_images.extend([os.path.join(subdir_path, img) for img in os.listdir(subdir_path)])

    # Combine the data and create labels
    images = bottle_images + canister_images + cans_images + cardboard_images + detergent_images
    labels = [0] * len(bottle_images) + [1] * len(canister_images) + [2] * len(cans_images) + [3] * len(cardboard_images) + [4] * len(detergent_images)
    # Define train directories for each class
    train_dirs = [train_bottle_dir, train_canister_dir, train_cans_dir, train_cardboard_dir, train_detergent_dir]
    # Move training data
    print(f"Moving {i} data...")
    move_files(images, labels, train_dirs)

    print(f"{i} data successfully moved to train directories.")
    


# Function to move files to train folder with progress bar
def move_files(file_paths, labels, train_dirs):
    with tqdm(total=len(file_paths), desc="Moving Files", bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt}") as pbar:
        for file_path, label in zip(file_paths, labels):
            if label == 0:  # bottle
                target_dir = train_dirs[0]
            elif label == 1:  # canister
                target_dir = train_dirs[1]
            elif label == 2:  # cans
                target_dir = train_dirs[2]
            elif label == 3:  # cardboard
                target_dir = train_dirs[3]
            elif label == 4:  # detergent
                target_dir = train_dirs[4]
            
            # Copy the file to the target directory
            shutil.copy(file_path, target_dir)
            pbar.update(1)  # Update progress bar by 1 step

In [3]:
data_prep("/kaggle/input/warp-waste-recycling-plant-dataset/Warp-C/train_crops","/kaggle/working/","train")

Moving train data...


Moving Files: 100%|██████████| 8823/8823

train data successfully moved to train directories.


In [4]:
data_prep("/kaggle/input/warp-waste-recycling-plant-dataset/Warp-C/test_crops","/kaggle/working/","test")

Moving test data...


Moving Files: 100%|██████████| 1551/1551

test data successfully moved to train directories.


In [5]:
# Create directories for the dataset
!mkdir -p /kaggle/working/train/images
!mkdir -p /kaggle/working/train/labels


In [6]:
import os
import shutil
from tqdm import tqdm

# Define source and destination paths
source_images = '/kaggle/input/warp-waste-recycling-plant-dataset/Warp-D/train/images/'
source_labels = '/kaggle/input/warp-waste-recycling-plant-dataset/Warp-D/train/labels/'
source_classes = '/kaggle/input/warp-waste-recycling-plant-dataset/Warp-D/classes.txt'

dest_images = '/kaggle/working/train/images/'
dest_labels = '/kaggle/working/train/labels/'
dest_classes = '/kaggle/working/classes.txt'

# Create destination directories if they don't exist
os.makedirs(dest_images, exist_ok=True)
os.makedirs(dest_labels, exist_ok=True)

# Copy images to the images directory
image_files = os.listdir(source_images)
with tqdm(total=len(image_files), desc="Copying Images", unit="file") as pbar:
    for filename in image_files:
        shutil.copy2(os.path.join(source_images, filename), os.path.join(dest_images, filename))
        pbar.update(1)  # Update the progress bar

# Copy labels to the labels directory
label_files = os.listdir(source_labels)
with tqdm(total=len(label_files), desc="Copying Labels", unit="file") as pbar:
    for filename in label_files:
        shutil.copy2(os.path.join(source_labels, filename), os.path.join(dest_labels, filename))
        pbar.update(1)  # Update the progress bar

# Copy classes.txt to the working directory
shutil.copy2(source_classes, dest_classes)

# Print confirmation
print("Files copied successfully!")


Copying Labels: 100%|██████████| 2452/2452 [00:12<00:00, 198.41file/s]

Files copied successfully!


In [7]:
import os
import shutil
from tqdm import tqdm

# Define source and destination paths
source_images = '/kaggle/input/warp-waste-recycling-plant-dataset/Warp-D/test/images'
source_labels = '/kaggle/input/warp-waste-recycling-plant-dataset/Warp-D/test/labels'

dest_images = '/kaggle/working/val/images/'
dest_labels = '/kaggle/working/val/labels/'

# Create destination directories if they don't exist
os.makedirs(dest_images, exist_ok=True)
os.makedirs(dest_labels, exist_ok=True)

# Copy images to the images directory
image_files = os.listdir(source_images)
with tqdm(total=len(image_files), desc="Copying Images", unit="file") as pbar:
    for filename in image_files:
        shutil.copy2(os.path.join(source_images, filename), os.path.join(dest_images, filename))
        pbar.update(1)  # Update the progress bar

# Copy labels to the labels directory
label_files = os.listdir(source_labels)
with tqdm(total=len(label_files), desc="Copying Labels", unit="file") as pbar:
    for filename in label_files:
        shutil.copy2(os.path.join(source_labels, filename), os.path.join(dest_labels, filename))
        pbar.update(1)  # Update the progress bar

# Copy classes.txt to the working directory
shutil.copy2(source_classes, dest_classes)

# Print confirmation
print("Files copied successfully!")

Copying Labels: 100%|██████████| 522/522 [00:01<00:00, 269.83file/s]

Files copied successfully!


In [8]:
# Read class names from the classes.txt file
with open("classes.txt", "r") as f:
    class_names = [line.strip() for line in f.readlines() if line.strip()]  # Avoid empty lines

# Determine the number of classes
num_classes = len(class_names)

# Create YAML file content
yaml_content = f"""
path: /kaggle/working/
train: /kaggle/working/train
val: /kaggle/working/val

nc: {num_classes}
names: {class_names}
"""

# Write the content to a YAML file
with open("dataset.yaml", "w") as file:
    file.write(yaml_content)

# Print the YAML content for verification
print("Generated YAML file content:")
print(yaml_content)


Generated YAML file content:

path: /kaggle/working/
train: /kaggle/working/train
val: /kaggle/working/val

nc: 28
names: ['bottle-blue', 'bottle-green', 'bottle-dark', 'bottle-milk', 'bottle-transp', 'bottle-multicolor', 'bottle-yogurt', 'bottle-oil', 'cans', 'juice-cardboard', 'milk-cardboard', 'detergent-color', 'detergent-transparent', 'detergent-box', 'canister', 'bottle-blue-full', 'bottle-transp-full', 'bottle-dark-full', 'bottle-green-full', 'bottle-multicolorv-full', 'bottle-milk-full', 'bottle-oil-full', 'detergent-white', 'bottle-blue5l', 'bottle-blue5l-full', 'glass-transp', 'glass-dark', 'glass-green']



In [ ]:
!pip install ultralytics

from ultralytics import YOLO

# Load the pre-trained YOLOv8 model
model = YOLO("yolov8n.pt")

# Start training
model.train(
    data="/kaggle/working/dataset.yaml",  # Path to your YAML file
    epochs=10,
    batch=16,
    imgsz=640,
    name="custom_yolo_model"
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 kB 33.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 269MB/s]

Ultralytics 8.3.35 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/dataset.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=custom_yolo_model, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=Fa

100%|██████████| 755k/755k [00:00<00:00, 31.1MB/s]
2024-11-21 15:31:59,292	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-21 15:31:59,783	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


...

val: New cache created: /kaggle/working/val/labels.cache


Plotting labels to runs/detect/custom_yolo_model/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000313, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to runs/detect/custom_yolo_model
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20       2.4G      1.516       4.83      1.325         37        640: 100%|██████████| 474/474 [01:31<00:00,  5.19it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:04<00:00,  4.17it/s]


                   all        522       1551      0.236      0.166     0.0651     0.0479

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/20      2.36G      1.372      3.935      1.255         42        640:  37%|███▋      | 175/474 [00:32<00:55,  5.43it/s]

In [ ]:
import glob 
from PIL import Image

In [ ]:
images=glob.glob('/kaggle/working/val/images/*')
results=model([images[0]],stream=False)
res=results[0].plot()
Image.fromarray(res)